# **SAESBOT;>**

In [1]:
#inicialización de estados
import re
import time
from datetime import datetime, date, timedelta

In [ ]:

Iniciar_Sesion_RE = re.compile(r"""
\b(?:
    iniciar\s+sesion |
    # Verbos comunes para acceder
    (?:entrar|acceder|ingresar|conect(?:ar|arme)|meter(?:me)?|abrir)\b
        (?:\s+(?:a|al|en))?(?:\s+(?:saes|portal|sistema))?
    |
    # Login/Sign in (incluye loguear/logear)
    log(?:in|\s*in|ue(?:ar|arse|arme)|e(?:ar|arse|arme)) |
    sign[-\s]*in
)\b
""", re.I | re.X)

Cerrar_Sesion_RE = re.compile(r"""
\b(?:
    cerrar\s+sesion |
    terminar\s+sesion |
    # Logout / Sign out
    logout | sign[-\s]*out |
    # Salir / desconectar
    (?:salir|desconect(?:ar|arme)) (?:\s+de\s+(?:mi\s+)?)?(?:cuenta|sesion)?
)\b
""", re.I | re.X)


# 2) Inscripción / Reinscripción
# Incluye acciones típicas: cargar/alta/seleccionar/elegir materias, turno/cita/ventana/ficha
Inscripcion_RE = re.compile(r"""
\b(?:
    # Nombres del proceso
    (?:pre)?inscrip(?:cion|cions)? | reinscrip(?:cion|cions)? |
    # Verbos
    reinscrib\w* | inscrib\w* |
    # Tareas asociadas a inscribir
    (?:cargar|alta\s+de|seleccion(?:ar)?|elegir)\s+materias? |
    # Ventanas / tiempos / fichas
    (?:horario|cita|turno|ventana|periodo|ficha)\s+de\s+(?:re)?inscrip(?:cion|cions)?
)\b
""", re.I | re.X)


# 3) Información personal, médica y de contacto
# Cubre “mis datos”, “perfil”, “actualizar/cambiar/modificar/editar” + campos típicos
Informacion_personal_RE = re.compile(r"""
\b(?:
    # Consultar / ver / editar datos
    (?:ver|consultar|revisar|checar|mostrar|editar|actualiz\w*|cambi\w*|modific\w*)\s+
       (?:mi\s+|mis\s+|la\s+)? (?:informacion|datos|perfil)
    |
    # Referencias directas a informacion personal/contacto/medica
    (?:informacion|datos|perfil)\s+(?:personal|de\s+contacto|medic[ao]s?)
    |
    # Campos específicos como disparadores
    (?:domicilio|direccion|telefono|celular|correo|email|curp|boleta|
       seguro(?:\s+facultativo)?|emergencia|alergias|tipo\s+de\s+sangre)
       (?:\s+(?:actualiz\w*|cambi\w*|modific\w*|ver|consultar))?
)\b
""", re.I | re.X)


# 4) Materias + Filtros (profesor/horario/cupo/clave/créditos/…)
# Incluye “materias/clases/asignaturas/grupos/oferta académica/horarios”
# y filtros: profesor/docente/maestro, cupo/lugares/disponibilidad, clave, créditos, plan, turno, salón, modalidad, semestre…
Materias_RE = re.compile(r"""
\b(?:
    # Acción + objeto
    (?:(?:ver|consultar|buscar|filtrar|encontrar|listar|mostrar)\s+)?
    (?:materias?|clases?|asignaturas?|grupos?|oferta\s+academica|horarios?)
    (?:\s+(?:disponibles?|oferta))?
    (?: .*?\b
        (?:profes?or(?:es)?|docentes?|maestros?|catedraticos?|titulares?|
           cupos?|lugares|espacios|disponibilidad|
           clave(?:\s+de\s+materia)?|creditos?|
           plan\s+de\s+estudios|optativas?|obligatorias?|
           modalidad|presencial|en\s*linea|mixta|
           turno|semestre|grupo|salon|aula)
    )?
)\b
""", re.I | re.X | re.U)


# 5) Trámites (dictamen, ETS, baja, servicio social, titulación, prácticas, becas, certificados, constancias…)
# Apunta a *tramitar/solicitar* o ver requisitos/formato/fechas de estos procesos.
Tramites_RE = re.compile(r"""
\b(?:
    # Palabra troncal + acciones
    tramites? (?:\s+(?:disponibles?|que\s+puedo\s+hacer))? |
    (?:ver|consultar|solicit\w*|tramitar|realiz\w*|hacer|agendar)\s+(?:los?\s+)?tramites?
    |
    # Claves de trámites
    dictamen(?:\s+(?:de\s+reingreso|de\s+equivalencia))? |
    (?:inscrib\w*\s+)?ets\b |
    baja\s+(?:temporal|definitiva) |
    servicio\s+social |
    titulaci[oó]n |
    practicas?\s+profesionales? |
    becas? |
    certificados? |
    constancias? |
    revalidaci[oó]n | equivalencias? |
    # Señales de intención de trámite
    (?:requisitos?|formato|formulario|cita|fechas?|convocatoria)\s+(?:para\s+)?(?:el\s+|la\s+|los\s+|las\s+)? 
       (?:dictamen|ets|baja|servicio\s+social|titulaci[oó]n|practicas?\s+profesionales?|becas?|certificados?|constancias?)
)\b
""", re.I | re.X)


# 6) Información académica (kardex, boleta, calificaciones, adeudos, horario)
Informacion_academica_RE = re.compile(r"""
\b(?:
    kardex |
    boleta(?:\s+(?:de\s+calificaciones|de\s+evaluacion))? |
    calificaciones?|califs?|notas? |
    promedio(?:\s+(?:general|acumulado))? |
    historial\s+academico |
    adeudos?|deudas? | pagos?\s+(?:pendientes?)? |
    faltas?|inasistencias? |
    # horarios como parte del perfil académico personal
    (?:mi\s+)?horarios? |
    # acciones típicas
    (?:ver|consultar|descargar|imprimir)\s+
       (?:kardex|boleta|calificaciones?|historial\s+academico|horarios?|adeudos?)
)\b
""", re.I | re.X)

In [ ]:
state=0
Salida=1
while Salida:
    if state==0:
        print("Hola soy el Chatbot de la FORD ¿En qué te puedo ayudar?")
        time.sleep(1)
        opcion=input("Soy capaz de informarte de nuestras promociones, venta de vehículos y ayudarte a agendar un cita. \n\t\t\t")
        if re.findall(Promo_RE, opcion, flags=0)!=[]:
            state=1
        elif re.findall(Cita_RE, opcion, flags=0)!=[]:
            state=2
        elif re.findall(Venta_RE, opcion, flags=0)!=[]:
            state=11
        elif re.findall(salir_RE, opcion, flags=0)!=[]:
            state=7
        else :
          state =8
    if state == 1:
        print("Nuestras promociones son...")
        state=6
    if state == 2:
        name = input("Dime tu nombre para agendar la cita. \n\t\t\t")
        state = 4
    if state == 3:
        print("En un momento te contactara un agente de ventas")
        state=6
    if state == 4:
        placa = input("Podrias proporcionarme tu placa. \n\t\t\t")
        if re.findall(placa_Re, placa, flags=0)!=[]:
            state = 9
        else:
            print("Placa Invalida")
    if state == 5:
        print("Gracias {} en un momento te atendera un operador".format(name))
        state=0
    if state == 6:
        opcion=input("¿te puedo ayudar en algo más?  \n\t\t\t")
        if re.findall(afirmacion_RE, opcion, flags=0!=[]):
            state = 0
        elif re.findall(salir_RE, opcion, flags=0!=[]):
            state = 7

        else:
            print("No pudimos procesar tu información seras redireccionado al menu de inicio")
            state = 0
    if state == 7:
      print("Gracias fue un placer atenderte")
      Salida=0
    if state == 8:
      print("La opción no pudo ser procesada")
      state=0
    if state == 9:
      print("Me podrias dar la fecha que deseas agendar, estamos disponibles los 7 dias")
      pes=29
      pmes=1
      pano=2020
      dia=int(input("Día"))
      mes=int(input("mes"))
      ano=int(input("Año"))
      fecha = datetime.now().date()
      fecha_dada = datetime(ano, mes, dia)
      fecha_final = fecha_dada.strftime('%Y-%m-%d')
      fecha_max = datetime(2020,5,30)
      fecha_tot = fecha_max.strftime('%Y-%m-%d')
      if str(fecha_final) == str(fecha):
        print("No puedes agendar cita para el día de hoy")
        state=9
      elif str(fecha_final) < str(fecha):
        print("No puedes agendar fechas pasadas")
        state=9
      elif str(fecha) > str(fecha_tot):
        print("Aún no tenemos fechas tan lejanas")
        state=9
      else:
        state=10
    if state ==10:
      print ("ahora me puedes dar la hora, estamos disponibles de 9:00 a 18:00,recuerda las citas duran 1 hora")
      hora=int(input("Hora"))
      if hora < 9 :
          print ("A esa hora aún no estamos en servicio")
          state=10
      elif hora>17:
          print ("A esa horano ya nos retiramos lo siento")
          state=10
      else:
          print ("Listo tus datos han sido tomados te comunicaremos al operador para completar la cita")
          state=3
    if state ==11:
      print("Me puedes dar tu correo electronico")
      correo=input()
      if re.match('^[(a-z0-9\_\-\.)]+@[(a-z0-9\_\-\.)]+\.[(a-z)]{2,15}$',correo.lower()):
        print ("Correo correcto")
        state=3
      else:
        print ("Correo incorrecto")
        state=11

